# Get the Top 5 Chunks to Check Paper's Stance

## Step 0: Environment Setup

In [16]:
from dotenv import load_dotenv # type: ignore
import os
from langchain_neo4j import Neo4jGraph # type: ignore
from libs import create_vector_index
import pandas as pd # type: ignore
from conn import connect2Googlesheet
from annolibs import get_all_chunks_per_paper, compare_embeddings
load_dotenv()

True

In [17]:
# Connect to Neo4j database
try:
    graph = Neo4jGraph(
        url=os.getenv("NEO4J_URL"),
        username=os.getenv("NEO4J_USERNAME"),
        password=os.getenv("NEO4J_PASSWORD")
    )
    print("Connected to Neo4j database successfully.")
except ValueError as e:
    print(f"Could not connect to Neo4j database: {e}")

# Check if the entities index exists
index_name = "entities"
query = "SHOW INDEXES YIELD name, type WHERE type = 'VECTOR' AND name = $index_name"

result = graph.query(query, params={"index_name": index_name})
if result:
    print("The 'entities' index already exists.")
else:
    create_vector_index(graph, "entities")

Connected to Neo4j database successfully.
The 'entities' index already exists.


## Step 1: Load Questions from Google Sheet

In [18]:
spreadsheet = connect2Googlesheet()

# Select the worksheet: relevance
worksheet = spreadsheet.get_worksheet(2)  

# Get all records as a list of dictionaries
data = worksheet.get_all_records()

# Convert to Pandas DataFrame
df_Paper = pd.DataFrame(data)
df_Paper.head()

,condition,number,docs,Question,Mahmud's Note,status,comments,
0,ARDS,1,ACURASYS,Does early administration of neuromuscular blocking agents increases the ventilator free days?,Like,,,
1,ARDS,2,ACURASYS,Do patients with severe ARDS being treated with neuromuscular blocking agents have increased muscle weakness?,Replace,fixed,,
2,ARDS,3,ROSE,"In patients with moderate to severe ARDS, does early use of continuous neuromuscular blockade improve mortality?","Maybe this question: In patients with moderate to severe ARDS, does early use of continuous neuromuscular blockade improve mortality?",fixed,,
3,ARDS,4,ROSE,Do patients with moderate-to-severe ARDS have a significance difference in mortality rate beween patients who recieved an early and continous cisatracurium infusion than those with usual care approach with lighter sedation targets?,Local question (not sure if this is the aim of your project) It will be nice as second step after proving the general summarization is working but focusing in general summarization would be priority in my opinion so you can have meanigful tool.,fixed,Wrong concept since PEEP by itself is mandatory component in ventilator.,"Does the use of neuromuscular blockers in patients with moderate-to-severe ARDS impact cardiovascular stability, particularly in terms of vasopressor requirements and hemodynamic effects, compared to sedation strategy without routine neuromuscular blockade?"
4,ARDS,5,FACTT,"Among patients with ALI/ARDS, does a conservative fluid management strategy improves lung function and decrease ventilator days compared to liberal strategy?","Local question (not sure if this is the aim of your project) consider (WikiJournal): In patients with ALI/ARDS that are intubated and receiving positive pressure ventilation, how does the conservative compare to the liberal fluid management strategy in reducing mortality?",fixed,Check if studies defined conservative by CVP < 4 or elese just dont mention how much the CVP (i prefer the last approach),


## Step 3: Get the chunks from each paper

In [19]:
# Usage example
#papers = sorted(list(set([doc.strip() for doc in df_Paper['docs'].tolist()])))

# Filter papers with 'Delirium' in condition
delirium_papers = df_Paper[df_Paper['condition'].str.contains('Delirium', case=False, na=False)]

# Display the filtered papers
print(f"Found {len(delirium_papers)} papers related to Delirium:")
display(delirium_papers)

# Get unique paper names
delirium_paper_names = delirium_papers['docs'].str.strip().unique()
# print(f"Number of unique paper names: {len(delirium_paper_names)}")
delirium_papers_chunks = get_all_chunks_per_paper(graph, delirium_paper_names)

Found 27 papers related to Delirium:


,condition,number,docs,Question,Mahmud's Note,status,comments,
23,Delirium,24,AID-ICU,Does treatment with haloperidol lead to a significantly greater number of days alive and out of the hospital at 90 days than placebo when encountering patients in ICU with delirium?,,,Can you check if that outcome used in the articles?,yes it was
24,Delirium,25,MIND-USA,"Does the use of ziprasidone as compared with placebo in patients with acute respiratory failure, shock, hypoactive or hyperactive delirium in the ICU alter the duration of delirium?",,,,
25,Delirium,26,SPICE-III,Is there more adverse events in ICU patients who recieved early dexmedetomidine?,,,,
26,Delirium,27,SPICE-III,"Among patients undergoing mechanical ventilation in the ICU, does early dexmedetomidine for sedation result in a mortality rate similar to that of the usual-care group?",,,"Make it two questions. one about is it enough as mono-therapy and second question about mortality (also u can make it broad and not necessarly 90 days, just mortality)",noted
27,Delirium,28,SEDCOM,Was there a difference between dexmedetomidine and midazolam in time at targeted sedation level in mechanically ventilated ICU patients?,,,,
28,Delirium,29,SEDCOM,Is bradycardia an adverse effect in critically ill patients receiving dexmedetomidine?,,,"Make it direct questions ""What is the side effects of dexa?",We are trying to make it yes and no questions
29,Delirium,30,SEDCOM,"At comparable sedation levels, did dexmedetomidine-treated patients spend less time on the ventilator, experience less delirium, and develop less tachycardia and hypertension?",,,consider making it multiple questions,
35,Delirium/Sepsis,36,DPSMVAS,is there a difference in outcomes in patients who receive dexmedetomidine than propofol among mechanically ventilated adults with sepsis?,,,,
36,ARDS/Delirium,37,ETSDMV,,,,,
37,ARDS/Delirium,38,SMDLTSWCI,,,,,


Found 100 chunks in paper AID-ICU
Found 100 chunks in paper MIND-USA
Found 100 chunks in paper SPICE-III
Found 100 chunks in paper SEDCOM
Found 0 chunks in paper DPSMVAS
Found 100 chunks in paper ETSDMV
Found 98 chunks in paper SMDLTSWCI
Found 100 chunks in paper EDvsLSMENDS
Found 100 chunks in paper DDMLSMVICP
Found 100 chunks in paper HDDSICU
Found 100 chunks in paper ESDDVCIP
Found 69 chunks in paper RDSCI
Found 100 chunks in paper DIICU
Found 100 chunks in paper RAEATICU
Found 100 chunks in paper AMTD
Found 100 chunks in paper AUDDICU
Found 100 chunks in paper AUPTICUPP
Found 100 chunks in paper RUSGATICUD
Found 94 chunks in paper ATD
Found 100 chunks in paper DPD
Found 100 chunks in paper ND
Found 59 chunks in paper PDHP
Found 99 chunks in paper ASD
Found 100 chunks in paper FDSD


In [20]:
# check if the expected number of papers match the actual number of papers
# Get list of expected papers
expected_papers = delirium_paper_names.tolist()

# Get list of actual papers from chunks_of_paper directory
actual_papers = [f.replace('chunks_of_', '').replace('.csv', '') 
                 for f in os.listdir('./chunks_of_paper') 
                 if f.endswith('.csv')]

# Find missing papers
missing_papers = set(expected_papers) - set(actual_papers)

print("Missing papers:")
for paper in missing_papers:
    print(f"- {paper}")

Missing papers:
- DPSMVAS


## Step 4: Compare Question Embedding and Paper Chunk Embeddings

In [34]:
# Set pandas display options to show the full text content
from annolibs import compare_embeddings


pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

idx = 10
test_question = df_Paper['Question'].iloc[idx]
print(f"Question {idx+1}: {test_question}")

for i, paper in enumerate(delirium_paper_names):
    print(f"\nPaper {i+1}: {paper}")
    paper_name = str(paper)  # without .pdf extension
    top5chunks = compare_embeddings(
        question=test_question,
        paper=paper_name,
        top_k=5
    )
    # Create DataFrame for GPT prompt
    prompt_df = pd.DataFrame({
        'question num': [idx+1],
        'question': [test_question],
        'paper': [paper],
        #'chunks': ['\n'.join([f"{i+1}. {text}" for i, text in enumerate(top5chunks['chunk_text'])])],
        'prompt': [f"Does the following question have any relevance to the text chunks below? Keep your answer short\n\nQuestion: {test_question}\n\nText Chunks:\n" + 
                '\n'.join([f"{i+1}. {text}" for i, text in enumerate(top5chunks['chunk_text'])])]
    })
    display(prompt_df)
# top5chunks = compare_embeddings(
#         question=test_question,
#         paper='ATD',
#         top_k=5
#     )
# display(top5chunks)

Question 11: Patients with septic shock undergoing mechanical ventilation, did continuous infusion of hydrocortisone result in lower 90-day mortality?

Paper 1: AID-ICU


,question num,question,paper,prompt
0,11,"Patients with septic shock undergoing mechanical ventilation, did continuous infusion of hydrocortisone result in lower 90-day mortality?",AID-ICU,"Does the following question have any relevance to the text chunks below? Keep your answer short\n\nQuestion: Patients with septic shock undergoing mechanical ventilation, did continuous infusion of hydrocortisone result in lower 90-day mortality?\n\nText Chunks:\n1. 25). With regard to the secondary outcomes, days alive without delirium or coma in the ICU and days alive without mechanical ventilation were analyzed with the use of a linear-regression model; the number of patients with one or more serious adverse reactions and the number of patients receiving rescue medication were ana­ lyzed with the use of logistic regression; and the total number of serious adverse reactions and the number of days with use of rescue medica\n2. randomization — no. (%) Mechanical ventilation‡ 320 (63.9) 305 (62.8) Vasopressors or inotropes§ 272 (54.3) 239 (49.2) Renal-replacement therapy¶ 77 (15.4) 70 (14.4) Predicted 90-day mortality — SMS-ICU score‖ 34.7±15.4 34.6±15.4 Delirium motor subtype at randomization —\n3. unit, and IQR interquartile range. †\tInformation on coexisting conditions was collected only for the prediction score (i.e., the Simplified Mortality Score for the Intensive Care Unit [SMS-ICU]); beginning on March 11, 2020, information on the presence of coronavirus disease 2019 (Covid-19) was also collected. ‡\tMechanical ventilation was defined as invasive or noninvasive mechanical ventilation, including use\n4. days, the number of days alive without mechanical ventilation at 90 days, the number of patients with one or more serious adverse reactions to haloperidol in the ICU, the total number of serious adverse reactions to haloperidol in the ICU, the number of patients receiving rescue medication, and the number of days with rescue medication per patient. We calculated the predicted 90-day mortality among the patients using the Simplified Mortality Score for\n5. calculated the predicted 90-day mortality among the patients using the Simplified Mortality Score for Intensive Care Unit (SMS-ICU), on which scores range from 0 to 42, with higher scores predicting higher 90-day mortality.14 Data were obtained from the patients’ medical records and hospital registries by the trial investigators or their delegates. Additional details about outcomes are provided in the Supplementary Appendix. Statistical Analysis Data from the AID-ICU"



Paper 2: MIND-USA


,question num,question,paper,prompt
0,11,"Patients with septic shock undergoing mechanical ventilation, did continuous infusion of hydrocortisone result in lower 90-day mortality?",MIND-USA,"Does the following question have any relevance to the text chunks below? Keep your answer short\n\nQuestion: Patients with septic shock undergoing mechanical ventilation, did continuous infusion of hydrocortisone result in lower 90-day mortality?\n\nText Chunks:\n1. days that a patient was alive and free from both de- lirium and coma during the 14-day intervention period). Secondary efficacy end points included duration of delirium, time to freedom from me- chanical ventilation (defined as extubation that was followed by at least a 48-hour period during which the patient was alive and free from me- chanical ventilation), time to final successful ICU discharge (defined as the last ICU discharge dur-\n2. delirium at randomization — no. (%) 161 (88) 172 (90) 172 (91) Diagnosis at admission — no. (%) Adult respiratory distress syndrome 39 (21) 44 (23) 35 (18) Sepsis 35 (19) 43 (22) 33 (17) Airway protection 53 (29) 46 (24) 44 (23) Chronic obstructive pulmonary disease, asthma, or other pulmonary disorder 23 (12) 20 (10) 28 (15)\n3. . **\t Shock was defined as treatment of hypotension with vasopressors or an intraaortic balloon pump. Table 1. Baseline Characteristics of the Trial Population.* The New England Journal of Medicine is produced by NEJM Group, a division of the Massachusetts Medical Society. Downloaded from nejm.org on February 13, 2025. For personal use only. No other uses without permission. Copyright © 2018 Massachusetts Medical Society. All rights reserved.\n4. odds ratios included unity for days with delirium and for days with coma during the 14-day intervention period (Fig. 2 and Table 2). In analyses of 30-day and 90-day survival as well as time to freedom from mechan- ical ventilation, ICU discharge, ICU readmission, and hospital discharge, the 95% confidence inter- vals for the hazard ratios of the effects of halo- peridol and ziprasidone\n5. ACHE) II is a prediction tool for death and measures severity of disease in the ICU; scores range from 0 to 71, with higher scores indicating greater severity of illness. ‖\t The Sequential Organ Failure Assessment (SOFA) is a tool to track organ failure in the ICU; scores range from 0 to 24, with higher scores indicating greater severity of illness. **\t Shock was defined as treatment of hypotension with vasopressors or an"



Paper 3: SPICE-III


...

,question num,question,paper,prompt
0,11,"Patients with septic shock undergoing mechanical ventilation, did continuous infusion of hydrocortisone result in lower 90-day mortality?",PDHP,"Does the following question have any relevance to the text chunks below? Keep your answer short\n\nQuestion: Patients with septic shock undergoing mechanical ventilation, did continuous infusion of hydrocortisone result in lower 90-day mortality?\n\nText Chunks:\n1. infection, an early factor proposed, stil1 seems likely.3,s In 1928, Doyle” noted that 9 of 28 patients with postoperative psychosis had an average leukocyte count of 16,900 and al1 28 patients experienced fever ranging from 100” F to 103” F during their psychotic episodes. Sep- sis with fever is stil1 a major element\n2. Po.stnne,rthrtic delirium: Olympia flurane 1% to 2% and nitrous oxide (N,O) 65% in 0, were given for maintenance with spontaneous assisted ventilation. During the 45-minute procedure, BP re- mained stable at approximately 100160 mmHg and HR ranged from 65 to 80 beats/minute. Ringer�\n3. stimulator were applied. (Pulse oximetry was not available at this time.) BP was 142/70 mmHg and heart rate (HR) was 60 beatsiminute with normal sinus rhythm. Moments after intravenous (IV) injection of 4.5 mg of d-tubocurare followed by 100 ug of fen- tanyl, the patient complained of a worsening head- \n4. urgical Patients. New York: John Wiley, 1958:359-60, 395. 15. Morse RM, Litin EM: Postoperative delirium: a study of etiological factors. Am J Psychiatvy 1969; 126:388-95. J. Clin. Anesth., vol. 3, JanuaryiFebruary 1991 63\n5. thesia care unit (PACU). Upon arrival in the PACU, the patient was re- sponsive; BP and HR were 130/80 mmHG and 80 beats/minute, respectively; and breathing was unla- bored. Within minutes, he complained of a headache and became agitated. Two bolus injections of butor-"



Paper 23: ASD


,question num,question,paper,prompt
0,11,"Patients with septic shock undergoing mechanical ventilation, did continuous infusion of hydrocortisone result in lower 90-day mortality?",ASD,"Does the following question have any relevance to the text chunks below? Keep your answer short\n\nQuestion: Patients with septic shock undergoing mechanical ventilation, did continuous infusion of hydrocortisone result in lower 90-day mortality?\n\nText Chunks:\n1. : prediction of mortality and 2-year functional outcomes. J Gerontol A Biol Sci Med Sci 55(9): M527–M534. Ely EW, Gautam S, Margolin R, et al. 2001. The impact of delirium in the intensive care unit on hospital length of stay. Intens Care Med 27(12): 1892–1900. Francis J, Martin D, Kapoor WN. 1990. A prospective study of delirium in\n2. which could have lead to overestimation of the correlation between the DOS Scale and the DRS-R-98. Another limitation of this study is that due to ﬂuctuations in symptoms of delirium during the hospital stay, symptoms were missed by physicians, since they saw patients mostly just once a day and based their score on different sources. Raters of the DRS-R-98 score were not blinded for the DOS Score, which could have lead to overestimation of the correlation\n3. the DRS-R-98 (Trzepacz et al., 1988). The DRS-R-98 has been demonstrated to be an adequate instrument for physicians to assess the severity of delirium (Meagher et al., 1998). The Dutch translation of the DRS-R-98 is a valid and reliable severity measure for delirium as demonstrated by a high inter-rater reliability and internal consistency (de Rooij et al., 2006). The DRS-\n4. to measure severity of delirium in two groups of elderly patients with a high risk of delirium, namely hip fracture and medical patients. The correlation between DRS-R-98 and the DOS Scale was 0.67; suggesting physician and nurses based measures of delirium severity were related. These results also suggest that repeated nurses’ observations can contribute to monitoring delirium severity in daily clinical practice. Correlations between DRS-R-98 and DOS Scale for hypoactive\n5. many missing items of the DRS-R-98 and DOS Scale. Another limitation of this study is that due to ﬂuctuations in symptoms of delirium during the hospital stay, symptoms were missed by physicians, since they saw patients mostly just once a day and based their score on different sources. Raters of the DRS-R-98 score were not blinded for the DOS Score, Figure 3 Scatter diagram DRS-R-98 and DOS Scale with a line of"



Paper 24: FDSD


,question num,question,paper,prompt
0,11,"Patients with septic shock undergoing mechanical ventilation, did continuous infusion of hydrocortisone result in lower 90-day mortality?",FDSD,"Does the following question have any relevance to the text chunks below? Keep your answer short\n\nQuestion: Patients with septic shock undergoing mechanical ventilation, did continuous infusion of hydrocortisone result in lower 90-day mortality?\n\nText Chunks:\n1. greater DRS-R98 syndromal status (w2 = 15.6, d.f.= 2, P50.001) but there was minimal relationship to other comorbidities. Similarly, the median duration of in-patient stay was longer with greater DRS-R98 delirium syndromal status (i.e. full syndromal delirium4subsyndromal delirium4no delirium, P= 0.006\n2. hospital over a 36 h period, and (b) compared the clinical profile of full and subsyndromal delirium defined according to both categorical and dimensional approaches. Method Study population The study was conducted over a 36 h weekend period at Cork University Hospital, a tertiary referral centre with 500 acute beds serving a 500 000 population in the South West of Ireland. All adult in-patients on the 15 May 2010 were considered for inclusion. Patients in the specialist units for psychiatry, intensive\n3. screened positive for possible delirium 133 assessed for delirium DSM-IV CAM DRS-R98 n = 47 Study refusal (n = 5) Severe aphasia (n = 23) Language barrier (n = 5) Too unwell (n = 6) Absent from ward n = 29 Patient discharge or transfer Absence on day leave Becoming gravely unwell 55 Delirium 78 No delirium 52 Delirium 41 Subsyndromal delirium\n4. ients on the 15 May 2010 were considered for inclusion. Patients in the specialist units for psychiatry, intensive care and isolation unit were not assessed. Patients were also excluded if they were comatose, deemed too unwell for interview by nursing staff; unable to communicate because of speech or language issues; or did not consent to study participation. Assessments The primary focus was to identify the frequency of delirium in a general hospital.23 Because DSM-IV criteria24 require inattention, patients\n5. (24) 4/28 (14) 28 (51) 7/48 (15) Number of active medical comorbidities, mean (s.d.) 2.5 (2.4) 2.5 (2.1) 2.2 (1.8) 2.8 (2.6) 1.8 (1.5) 2.3 (1.9) 2.5 (2.5 2.3 (1.9) Duration of in"


In [22]:
top5chunks = compare_embeddings(
        question=test_question,
        paper='AID-ICU',
        top_k=5
    )
gpt_prompt = f"Does question '{test_question}' have any relevance to '{[_ for _ in top5chunks['chunk_text']]}'? Answer yes or no"
print(gpt_prompt)

Does question 'Does early administration of neuromuscular blocking agents increases the ventilator free days?' have any relevance to '[' ‡\t\x07Mechanical ventilation was defined as invasive or noninvasive mechanical ventilation, including use of a continuous  positive airway pressure (CPAP) mask or CPAP by means of tracheostomy within 24 hours before randomization.  Intermittent CPAP was not defined as mechanical ventilation. §\t\x07Use of vasopressors or inotropes was defined as continuous infusion of norepinephrine, epinephrine, phenylephrine,  vasopressin', ' randomization — no. (%) Mechanical ventilation‡ 320 (63.9) 305 (62.8) Vasopressors or inotropes§ 272 (54.3) 239 (49.2) Renal-replacement therapy¶ 77 (15.4) 70 (14.4) Predicted 90-day mortality — SMS-ICU score‖ 34.7±15.4 34.6±15.4 Delirium motor subtype at randomization —', ' or Restraint in the ICU  after Randomization. Variable Haloperidol (N\u2009=\u2009501) Placebo (N\u2009=\u2009486) Median duration of trial intervention (

In [23]:
# Create DataFrame for GPT prompt
prompt_df = pd.DataFrame({
    'question num': [idx+1],
    'question': [test_question],
    'paper': [paper],
    'chunks': ['\n'.join([f"{i+1}. {text}" for i, text in enumerate(top5chunks['chunk_text'])])],
    'prompt': [f"Does the following question have any relevance to the text chunks below?\n\nQuestion: {test_question}\n\nText Chunks:\n" + 
              '\n'.join([f"{i+1}. {text}" for i, text in enumerate(top5chunks['chunk_text'])])]
})
